#### This code converts fluid data to graph data into a format that aligns with PyTorch Geometric format. It has the following attributes:
 - **data.x**: Node feature matrix with shape [num_nodes, num_node_features]
 - **data.edge_index**: Graph connectivity in COO format with shape [2, num_edges] and type torch.long
 - **data.y**: Reynolds number associated with each data points with shape [1] and type torch.long
 - **data.pos**: Node position matrix with shape [num_nodes, num_dimensions]

In [1]:
# Import libraries
import numpy as np
import torch
from torch_geometric.data import Data

#### Load the values of x-component of velocity vector from FEM solution of Navier-Stokes equation for 4000 time points and different Reynolds number. 

In [3]:
# data loader
data_folder = "./datasets"

# For Reynolds number varying from 20,21,...40 and 100,101,..,120
data_file = '/vel_x.npy'

# For Reynolds number varying from 30,40,...,120
#data_file= '/vel_x_fluid_MNIST.npy'

if data_file == '/vel_x.npy':
    list1 = list(range(20,41,1))
    list2 = list(range(100,121,1))
    Relist = list1 + list2    
else:
    Relist = list(range(30,121,10))


data_filename = data_folder + data_file 

vel_x = np.load(data_filename)

graphs_all = vel_x.reshape(1,vel_x.shape[0]*vel_x.shape[1],vel_x.shape[2])

labels = np.repeat(Relist,4000)

Graph_Data_Size = graphs_all.shape[1]
#print(Graph_Data_Size)

In [4]:
# Define function to convert fluid array to graph
def load_flow_graph(data_size=4000):
    data_list = []
    edge = torch.tensor(np.genfromtxt(data_folder + '/refined_edges', dtype=np.int32), dtype=torch.long)
    pos = torch.tensor(np.load(data_folder + '/position.npy'), dtype=torch.float)
    for t in range(data_size): 
        x = torch.tensor(graphs_all[:,t,:].T, dtype=torch.float) 
        y = torch.tensor(labels[t], dtype=torch.long)
        d = Data(x=x, edge_index=edge.t().contiguous(), y=y, pos=pos)
        data_list.append(d)
    print("Complete!")
    return data_list

In [5]:
# Convert fluid data into PyTorch Geometric graph dataset format
flow_list = load_flow_graph(data_size = Graph_Data_Size)

# Save the graph data
torch.save(flow_list, 'vel_x_graph.pt')

Complete!
